In [1]:
import numpy as np
import get_data
from scipy.optimize import fsolve
from scipy.optimize import minimize
import math

In [2]:
data = np.genfromtxt(
        r".\data\01_data_mars_opposition_updated.csv",
        delimiter=",",
        skip_header=True,
        dtype="int",
    )

In [3]:
times = get_data.get_times(data)
oppositions = get_data.get_oppositions(data)

In [4]:
print("Times: ", times)
print("Oppositions: ", oppositions)

Times:  [   0.          770.10208333 1534.73819444 2299.24444444 3069.20277778
 3854.25833333 4663.66388889 5459.96388889 6234.59236111 7000.52152778
 7764.52916667 8531.61944444]
Oppositions:  [[ 6.64763889e+01  1.66666667e+00]
 [ 1.06925000e+02  4.10000000e+00]
 [ 1.41602778e+02  4.53333333e+00]
 [ 1.75716667e+02  3.68333333e+00]
 [ 2.14383333e+02  1.20000000e+00]
 [ 2.66716667e+02 -4.00000000e+00]
 [ 3.42266667e+02 -6.03333333e+00]
 [ 4.75277778e+01  1.33333333e-01]
 [ 9.24666667e+01  3.55000000e+00]
 [ 1.28633333e+02  4.50000000e+00]
 [ 1.62450000e+02  4.16666667e+00]
 [ 1.98619444e+02  2.43333333e+00]]


In [5]:
# def equations(t, x1, y1, dx, dy, h, k, r):
#     return (x1 + t * dx - h)**2 + (y1 + t * dy - k)**2 - r**2

# # # Define the ray function
# def ray(x1, y1, dx, dy, t):
#     return x1 + t * dx, y1 + t * dy

# def equation2(x, c, e_1, e_2, z, r):
#     return (x - np.cos(c)) ** 2 + (e_1 * np.sin(e_2) - np.sin(c) + x * np.tan(z) - e_1*np.cos(e_2)*np.tan(z)) ** 2 - r ** 2 

def y_intersection(e_1, e_2, z, x):
    return e_1 * math.sin(e_2) + math.tan(z) * x - math.tan(z) * e_1 * math.cos(e_2)

def quad_form(e_1, e_2, c, z, r):
    a = 1 + math.tan(z)**2
    b = -2 * math.cos(c) + 2 * math.tan(z) * (e_1 * math.sin(e_2) - math.sin(c) - e_1 * math.cos(e_2) * math.tan(z))
    ci = math.cos(c)**2 + (e_1 * math.sin(e_2) - math.sin(c) - e_1 * math.cos(e_2) * math.tan(z))**2 - r**2
    discriminant = b**2 - 4*a*ci

    if discriminant < 0:
        raise ValueError("No real roots")
    
    root1 = (-b + math.sqrt(discriminant))/ (2*a)
    root2 = (-b - math.sqrt(discriminant))/ (2*a)
    
    return root1, root2

In [7]:
def Erros_and_MaxError(x):
    c, r, e1, e2, z, s = x
    errors = []
    c_rad = math.radians(c)
    e2_rad = math.radians(e2)
    equant_longitudes = []

    for i in times:
        equant_long = z + s * (i - times[0])
        equant_long = equant_long - 360 * math.floor(equant_long / 360)
        equant_long = math.radians(equant_long)
        equant_longitudes.append(equant_long)
    
    predicted_angles = []

    for equant_long in equant_longitudes:
        predicted_x1, predicted_x2 = quad_form(e1, e2_rad, c_rad, equant_long, r)
        predicted_y1, predicted_y2 = y_intersection(e1, e2_rad, equant_long, predicted_x1), y_intersection(e1, e2_rad, equant_long, predicted_x2)
        if predicted_y1 > predicted_y2:
            if 0 <= equant_long <= math.pi:
                predicted_x = predicted_x1
                predicted_y = predicted_y1
            else:
                predicted_x = predicted_x2
                predicted_y = predicted_y2
        else:
            if 0 <= equant_long <= math.pi:
                predicted_x = predicted_x2
                predicted_y = predicted_y2
            else:
                predicted_x = predicted_x1
                predicted_y = predicted_y1

        if predicted_x == 0:
            raise ValueError("Divide by 0 Error")
        if predicted_x > 0 and predicted_y > 0:
            predicted_angle = math.degrees(math.atan(predicted_y / predicted_x))
        elif predicted_x < 0 and predicted_y > 0:
            predicted_angle = 180 + math.degrees(math.atan(predicted_y / predicted_x))
        elif predicted_x < 0 and predicted_y < 0:
            predicted_angle = 180 + math.degrees(math.atan(predicted_y / predicted_x))
        else:
            predicted_angle = 360 + math.degrees(math.atan(predicted_y / predicted_x))
        predicted_angles.append(predicted_angle)
    
    errors = []
    for j in range(len(predicted_angles)):
        errors.append(predicted_angles[j] - oppositions[j][0])

    max_error = 0
    for k in errors:
        if abs(k) > abs(max_error):
            max_error = k
    return errors, max_error


In [8]:
def MarsEqantOrbit(x):
    c, r, e1, e2, z, s = x
    errors = []
    c_rad = math.radians(c)
    e2_rad = math.radians(e2)
    equant_longitudes = []

    for i in times:
        equant_long = z + s * (i - times[0])
        equant_long = equant_long - 360 * math.floor(equant_long / 360)
        equant_long = math.radians(equant_long)
        equant_longitudes.append(equant_long)
    
    predicted_angles = []

    for equant_long in equant_longitudes:
        predicted_x1, predicted_x2 = quad_form(e1, e2_rad, c_rad, equant_long, r)
        predicted_y1, predicted_y2 = y_intersection(e1, e2_rad, equant_long, predicted_x1), y_intersection(e1, e2_rad, equant_long, predicted_x2)
        if predicted_y1 > predicted_y2:
            if 0 <= equant_long <= math.pi:
                predicted_x = predicted_x1
                predicted_y = predicted_y1
            else:
                predicted_x = predicted_x2
                predicted_y = predicted_y2
        else:
            if 0 <= equant_long <= math.pi:
                predicted_x = predicted_x2
                predicted_y = predicted_y2
            else:
                predicted_x = predicted_x1
                predicted_y = predicted_y1

        if predicted_x == 0:
            raise ValueError("Divide by 0 Error")
        if predicted_x > 0 and predicted_y > 0:
            predicted_angle = math.degrees(math.atan(predicted_y / predicted_x))
        elif predicted_x < 0 and predicted_y > 0:
            predicted_angle = 180 + math.degrees(math.atan(predicted_y / predicted_x))
        elif predicted_x < 0 and predicted_y < 0:
            predicted_angle = 180 + math.degrees(math.atan(predicted_y / predicted_x))
        else:
            predicted_angle = 360 + math.degrees(math.atan(predicted_y / predicted_x))
        predicted_angles.append(predicted_angle)
    
    errors = []
    for j in range(len(predicted_angles)):
        errors.append(predicted_angles[j] - oppositions[j][0])

    predicted = np.array(predicted_angles)
    actual = np.array([oppositions[k][0] for k in range(len(oppositions))])
    squared_error = np.sum((predicted - actual)**2)
    return squared_error

In [19]:
def bestMarsOrbitParams(times, oppositions):
    '''Returns the best parameters for the Mars orbit by doing an exhaustive search'''
    # [c, r, e1, e2, z, s]
    initial_guess = [0, 0, 0, 0, 0, 0]
    times = times
    oppositions = oppositions
    s = 0.5241
    min_error = 100
    min_errors_list = []
    tolerance = 0
    r_out, s_out, c_out, e1_out, e2_out, z_out = 0, 0, 0, 0, 0, 0
    for r in np.linspace(1, 10, 10):
        for c in np.linspace(0, 360, 18):
            for e1 in np.linspace(1, r - 0.01, 10):
                for e2 in np.linspace(0, 360, 18):
                    for z in np.linspace(0, 360, 18):
                        try:
                            result = minimize(MarsEqantOrbit, [c, r, e1, e2, z, s], method='powell')
                            errors, maxError = Erros_and_MaxError(list(result.x))
                            if abs(maxError) < 0.06 and abs(maxError) <= abs(min_error):
                                min_error = maxError
                                min_errors_list = errors
                                c_out, r_out, e1_out, e2_out, z_out, s_out = list(result.x)     
                                print(f"r = {r}, s = {s}, c = {c}, e1 = {e1}, e2 = {e2}, z = {z}, errors = {errors}, maxError = {maxError}")
                                print(f"r_out = {r_out}, s_out = {s_out}, c_out = {c_out}, e1_out = {e1_out}, e2_our = {e2_out}, z_out = {z_out}, min_errors_list = {min_errors_list}, min_error = {min_error}")
                                tolerance = 0
                            else:
                                tolerance += 1
                            if tolerance > 10000:
                                    return r_out, s_out, c_out, e1_out, e2_out, z_out, min_errors_list, min_error
                        except:
                            continue
    return r_out, s_out, c_out, e1_out, e2_out, z_out, min_errors_list, min_error

In [20]:
r, s, c, e1, e2, z, errors, maxError = bestMarsOrbitParams(
    times, oppositions
)
# print("BESTTTTTTTT ++++++ r = {:.4f}, s = {:.4f}, c = {:.4f}, e1 = {:.4f}, e2 = {:.4f}, z = {:.4f}, errors = {:.4f}, maxError = {:.4f}".format(r, s, c, e1, e2, z, errors, maxError))

r = 1.0, s = 0.5241, c = 0.0, e1 = 1.0, e2 = 42.35294117647059, z = 127.05882352941177, errors = [-0.03850397277216189, 0.01245371053163069, 0.02007102331018018, -0.006523968927382384, 0.0291810797175458, -0.01497234337671216, 0.0060700517216787375, 0.018329512784482915, 0.007847269498839182, 0.003374246548418114, -0.03486490386185892, -0.002339765974312513], maxError = -0.03850397277216189
r_out = 8.781247552976625, s_out = 0.5240814517028298, c_out = 148.9484369384784, e1_out = 1.6314087874881058, e2_our = 148.92930635263534, z_out = 55.83746369573622, min_errors_list = [-0.03850397277216189, 0.01245371053163069, 0.02007102331018018, -0.006523968927382384, 0.0291810797175458, -0.01497234337671216, 0.0060700517216787375, 0.018329512784482915, 0.007847269498839182, 0.003374246548418114, -0.03486490386185892, -0.002339765974312513], min_error = -0.03850397277216189
r = 1.0, s = 0.5241, c = 211.76470588235293, e1 = 0.991111111111111, e2 = 169.41176470588235, z = 63.529411764705884, error

In [11]:
print(r, s, c, e1, e2, z, errors, maxError)

8.675199386469073 0.5240810650333936 148.92895420324808 1.6120197451470566 148.9318624984065 55.8389212681124 [-0.035706662541514333, 0.008367130610537288, 0.019686529582031653, -0.001140502720488712, 0.03149420503592637, -0.024783850906203497, 0.009077761891205682, 0.023529392025054108, 0.0029609002745019097, -0.0011668646487805745, -0.0330643882880679, 0.0007198958661263077] -0.035706662541514333


In [12]:
print(Erros_and_MaxError([169.41176470588235, 2.0, 1.99, 127.05882352941177, 42.35294117647059, 0.5241]))

([38.87694898966258, 11.08923775550656, -14.628093633288671, -35.59304611309025, -44.72565400257386, -40.608917944413804, -335.1937108250888, 48.3366944724203, 21.730431020887153, -5.149253003535847, -28.433739746548838, -43.031740085938964], -335.1937108250888)


In [13]:
var = minimize(MarsEqantOrbit, [148, 8.3, 1.53, 148, 55, 0.5241], method="powell")
# print(list(var.x))
print(var)


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.004825371929075399
       x: [ 1.489e+02  8.782e+00  1.632e+00  1.489e+02  5.584e+01
            5.241e-01]
     nit: 17
   direc: [[-5.579e-02  1.840e-01 ... -3.941e-04  1.607e-06]
           [-3.808e-02  1.621e-01 ... -9.841e-03  7.163e-07]
           ...
           [-2.374e+01 -9.049e-02 ...  2.091e-01 -1.468e-06]
           [ 1.232e-02  3.173e-03 ...  3.768e-04 -1.665e-07]]
    nfev: 1119


In [14]:
print(Erros_and_MaxError(list(var.x)))

([-0.03862206514234856, 0.012569568930501873, 0.020253542373239952, -0.006529128711491694, 0.028952177698045034, -0.015037485061839106, 0.006165402557883226, 0.018156780105343273, 0.007866132480231158, 0.0035549499897342685, -0.034785215213332776, -0.002513591000621318], -0.03862206514234856)


In [190]:
print(sajdhfadsjkhfjkadshfkjaserrors([148, 8.2, 1.53, 148, 55, 0.5241]))

([-0.833438075566022, -0.8300544193568697, -0.8129183108358689, -0.8108639785334049, -0.7794864974327425, -0.8469764314899635, -0.6774906483511813, -0.6386615190639588, -0.7260193534535233, -0.7483269462380235, -0.7592596169581611, -0.7076371420461101], 0.8469764314899635)


In [32]:
import numpy as np
from scipy.optimize import fsolve

# Example values for center_of_orbit, equant_coords, and equant_longitudes
center_of_orbit = (0, 0)
equant_coords = (1, 1)
equant_longitudes = np.linspace(0, 2 * np.pi, 10)  # Example values, replace with actual longitudes
predicted_positions = []

# Parameters of the orbit
h, k, r = center_of_orbit[0], center_of_orbit[1], 1  # Example radius_of_orbit
# Starting point of ray
x1, y1 = equant_coords[0], equant_coords[1]
# Direction of ray will be defined in the loop
initial_guess = [0]

for equant_long in equant_longitudes:
    ray_direction = np.array([np.cos(equant_long), np.sin(equant_long)])  # Correctly create the array
    # Find the intersection of the line with the orbit
    t_solutions = fsolve(equations, initial_guess, args=(x1, y1, ray_direction[0], ray_direction[1], h, k, r))
    intersection_point_x, intersection_point_y = ray(x1, y1, ray_direction[0], ray_direction[1], t_solutions[0])
    predicted_angle = np.degrees(np.arctan2(intersection_point_y, intersection_point_x))
    predicted_positions.append(predicted_angle)

print(predicted_positions)

# Example implementation of get_errors function
def get_errors(params):
    # Example implementation, replace with your actual logic
    return [param * 2 for param in params]

# Call the get_errors function with the specified parameters
errors = get_errors([148, 8.2, 1.53, 148, 55, 0.5241])

# Print the output
print(errors)

[89.99999945045343, 47.08010123473993, 25.790619735249706, 29.993284741195147, 70.00597935398527, 56.70338236351071, 38.52929856756008, 10.00180949603043, 49.99076144061323, 89.99999979098185]
[296, 16.4, 3.06, 296, 110, 1.0482]


C:\Users\aadhi\AppData\Local\Temp\ipykernel_1964\581368429.py:20: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  t_solutions = fsolve(equations, initial_guess, args=(x1, y1, ray_direction[0], ray_direction[1], h, k, r))


In [98]:
def get_errors(x):
    """Calculates the discrepancies for the given parameters."""
    c, r, e1, e2, z, s = x
    errors = []
    center_of_orbit = (np.cos(c), np.sin(c))
    equant_coords = (e1*np.cos(e2), e1*np.sin(e2))
    equant_longitudes = []
    for i in times:
        equant_long = z + s * (i - times[0])
        equant_longitudes.append(equant_long)
    equant_longitudes = np.array(equant_longitudes)
    
    predicted_positions = []

    # Parameters of the orbit
    h, k = center_of_orbit[0], center_of_orbit[1] # Example radius_of_orbit
    # Starting point of ray
    x1, y1 = equant_coords[0], equant_coords[1]
    # Direction of ray will be defined in the loop
    initial_guess = [0]

    for equant_long in equant_longitudes:
        ray_direction = np.array([np.cos(equant_long), np.sin(equant_long)])
        # Find the intersection of the line with the orbit
        t_solutions = fsolve(equations, initial_guess, args=(x1, y1, ray_direction[0], ray_direction[1], h, k, r))
        intersection_point_x, intersection_point_y = ray(x1, y1, ray_direction[0], ray_direction[1], t_solutions[0])
        predicted_angle = np.degrees(np.arctan2(intersection_point_y, intersection_point_x))
        predicted_positions.append(predicted_angle)
    # print(predicted_positions)
    # for i in range(len(times)):
    #     equant_long = z + s * (times[i] - times[0])
    #     opposition_angle = oppositions[i][0]
    #     ray_direction = np.array(np.cos(equant_long), np.sin(equant_long))
    #     initial_guess = [0]
    #     t_solutions = fsolve(equations, initial_guess, args=(equant_coords[0], equant_coords[1], ray_direction[0], ray_direction[1], center_of_orbit[0], center_of_orbit[1], r))
    #     intersection_point_x, intersection_point_y = ray(equant_coords[0], equant_coords[1], ray_direction[0], ray_direction[1], t_solutions[0])
    #     predicted_angle = np.degrees(np.arctan2(intersection_point_y, intersection_point_x))
    #     errors.append(predicted_angle - opposition_angle)
    errors = []
    for i in range(len(predicted_positions)):
        if predicted_positions[i] < 0:
            # number_of_rotations = int(-predicted_positions[i] / 360) + 1
            predicted_positions[i] += 360
        error = predicted_positions[i] - oppositions[i][0]
        errors.append(error)
    # print(errors)
    # max_error = 0
    # for j in errors:
    #     if abs(j) > max_error:
    #         max_error = abs(j)
    
    predicted = np.array(predicted_positions)
    actual = np.array([oppositions[i][0] for i in range(len(oppositions))])
    squared_error = np.sum((predicted - actual)**2)
    return squared_error

In [122]:
def something(x):
    c, r, e1, e2, z, s = x
    c = np.radians(c)
    e2 = np.radians(e2)
    errors = []
    center_of_orbit = (np.cos(c), np.sin(c))
    equant_coords = (e1*np.cos(e2), e1*np.sin(e2))
    equant_longitudes = []
    for i in times:
        equant_long = z + s * (i - times[0])
        equant_long = np.radians(equant_long)
        equant_longitudes.append(equant_long)
    equant_longitudes = np.array(equant_longitudes)
    print(f'equant_longitudes = {equant_longitudes}')
    predicted_positions = []

    # Parameters of the orbit
    h, k = center_of_orbit[0], center_of_orbit[1] # Example radius_of_orbit
    # Starting point of ray
    x1, y1 = equant_coords[0], equant_coords[1]
    # Direction of ray will be defined in the loop
    initial_guess = [0]

    for equant_long in equant_longitudes:
        predicted_x = quad_form(e1, e2, c, equant_long, r)
        predicted_y = equation3(e1, e2, equant_long, predicted_x)
        predicted_angle = np.degrees(np.arctan2(predicted_y, predicted_x))
        predicted_positions.append(predicted_angle)
        '''x_guess = 0
        x_solution = fsolve(equation2, x_guess, args=(c, e1, e2, z, r))
        predicted_x = x_solution[0]
        predicted_y = equation3(e1, e2, z, predicted_x)
        predicted_angle = np.degrees(np.arctan2(predicted_y, predicted_x))
        predicted_positions.append(predicted_angle)'''
        # ray_direction = [np.cos(equant_long), np.sin(equant_long)]
        # Find the intersection of the line with the orbit
        # t_solutions = fsolve(equations, initial_guess, args=(x1, y1, ray_direction[0], ray_direction[1], h, k, r))
        # t_solutions = [t for t in t_solutions if t >= 0]
        # if t_solutions:
        #     t = min(t_solutions)
        #     intersection_point_x, intersection_point_y = ray(x1, y1, ray_direction[0], ray_direction[1], t)
        #     predicted_angle = np.degrees(np.arctan2(intersection_point_y, intersection_point_x))
        #     predicted_positions.append(predicted_angle)
        # print(f't_solutions = {t_solutions}')
        # intersection_point_x, intersection_point_y = ray(x1, y1, ray_direction[0], ray_direction[1], t_solutions[0])
        # print(f'intersection_point_x = {intersection_point_x}')
        # print(f'intersection_point_y = {intersection_point_y}')
        # predicted_angle = np.degrees(np.arctan2(intersection_point_y, intersection_point_x))
        # predicted_positions.append(predicted_angle)
    print(f'predicted_positions = {predicted_positions}')
    # for i in range(len(times)):
    #     equant_long = z + s * (times[i] - times[0])
    #     opposition_angle = oppositions[i][0]
    #     ray_direction = np.array(np.cos(equant_long), np.sin(equant_long))
    #     initial_guess = [0]
    #     t_solutions = fsolve(equations, initial_guess, args=(equant_coords[0], equant_coords[1], ray_direction[0], ray_direction[1], center_of_orbit[0], center_of_orbit[1], r))
    #     intersection_point_x, intersection_point_y = ray(equant_coords[0], equant_coords[1], ray_direction[0], ray_direction[1], t_solutions[0])
    #     predicted_angle = np.degrees(np.arctan2(intersection_point_y, intersection_point_x))
    #     errors.append(predicted_angle - opposition_angle)
    errors = []
    for i in range(len(predicted_positions)):
        if predicted_positions[i] < 0:
            # number_of_rotations = int(-predicted_positions[i] / 360) + 1
            predicted_positions[i] += 360
        error = predicted_positions[i] - oppositions[i][0]
        print(f'predicted_positions[i] = {predicted_positions[i]}')
        print(f'oppositions[i][0] = {oppositions[i][0]}')
        print(f'error = {error}')
        errors.append(error)
    # print(f'after predicted_positions = {predicted_positions}')
    # print(f'errors = {errors}')
    max_error = 0
    for j in errors:
        if abs(j) > max_error:
            max_error = abs(j)
    return errors, max_error

In [123]:
print(something([148, 8.2, 1.53, 148, 55, 0.5241]))

equant_longitudes = [ 0.95993109  8.00426324 14.99859667 21.99174222 29.03475945 36.21587506
 43.61972671 50.9036983  57.98943456 64.99559591 71.98418053 79.00096289]
predicted_positions = [40.92245477297909, 96.28576431041229, -38.25874334317069, -3.7610564407275398, 32.90701494886947, 93.38969172091397, -21.554396494857027, 27.27622016586421, -95.16915122650961, -50.274216932730226, -17.152763711991305, 18.795209031818263]
predicted_positions[i] = 40.92245477297909
oppositions[i][0] = 66.47638888888889
error = -25.553934115909804
predicted_positions[i] = 96.28576431041229
oppositions[i][0] = 106.92500000000001
error = -10.639235689587721
predicted_positions[i] = 321.7412566568293
oppositions[i][0] = 141.60277777777776
error = 180.13847887905155
predicted_positions[i] = 356.23894355927246
oppositions[i][0] = 175.71666666666667
error = 180.5222768926058
predicted_positions[i] = 32.90701494886947
oppositions[i][0] = 214.38333333333333
error = -181.47631838446387
predicted_positions[i] =

In [160]:
print(sajdhfadsjkhfjkadshfkjaserrors([229.29486885, 20.02190621, 2.7866114, 217.48526987, 56.47213594,-1.09393048]))

Predicted angle:  59.20198448120655
Actual angle:  66.47638888888889
Predicted angle:  121.94101808168354
Actual angle:  106.92500000000001
Predicted angle:  172.3037713001574
Actual angle:  141.60277777777776
Predicted angle:  64.63808978403576
Actual angle:  175.71666666666667
Predicted angle:  126.98477149945818
Actual angle:  214.38333333333333
Predicted angle:  153.36406679237493
Actual angle:  266.71666666666664
Predicted angle:  7.811773736916643e-06
Actual angle:  342.26666666666665
Predicted angle:  25.475310791551994
Actual angle:  47.52777777777778
Predicted angle:  81.48525506357183
Actual angle:  92.46666666666667
Predicted angle:  146.20561771697726
Actual angle:  128.63333333333333
Predicted angle:  24.575192338196565
Actual angle:  162.45
Predicted angle:  89.45036415200053
Actual angle:  198.61944444444444
([-7.27440440768234, 15.016018081683526, 30.700993522379633, -111.07857688263091, -87.39856183387515, -113.35259987429171, -342.2666588548929, -22.052466986225784, -